<a href="https://colab.research.google.com/github/harshith7304/carrental/blob/main/gan_with_source_bert_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 nltk sentence-transformers faiss-cpu transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyli

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [4]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import faiss
import torch
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
def fetch_website_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = ' '.join(p.get_text() for p in soup.find_all('p'))
    return text, url

In [6]:
def split_into_chunks(text, chunk_size=1000):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []

    current_length = 0
    for sentence in sentences:
        if current_length + len(sentence) > chunk_size:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(sentence)
        current_length += len(sentence)

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks


In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def embed_text(chunks):
    embeddings = model.encode(chunks, batch_size=32, show_progress_bar=True)
    return embeddings


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
def store_embeddings(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

In [9]:
def retrieve_relevant_chunks(query, index, chunks, urls, top_k=3):
    query_embedding = model.encode([query])
    _, indices = index.search(query_embedding, top_k)
    relevant_chunks = [(chunks[idx], urls[idx]) for idx in indices[0]]
    return relevant_chunks


In [10]:
qa_pipeline = pipeline('question-answering', model='deepset/roberta-large-squad2')

def answer_question(query, relevant_chunks):
    answers = []
    for chunk, _ in relevant_chunks:
        result = qa_pipeline(question=query, context=chunk)
        answers.append(result)

    if answers:
        best_answer = max(answers, key=lambda x: x['score'])
        return best_answer['answer']
    else:
        return "No relevant answer found."

def rag_pipeline(urls, query):

    with ThreadPoolExecutor() as executor:
        contents = list(executor.map(fetch_website_content, urls))

    all_chunks = []
    all_urls = []
    for content, url in contents:
        chunks = split_into_chunks(content)
        all_chunks.extend(chunks)
        all_urls.extend([url] * len(chunks))

    embeddings = embed_text(all_chunks)
    index = store_embeddings(embeddings)
    relevant_chunks = retrieve_relevant_chunks(query, index, all_chunks, all_urls)

    answer = answer_question(query, relevant_chunks)
    sources = set(url for _, url in relevant_chunks)

    return answer, sources



config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [11]:
urls = [
    "https://gameofthrones.fandom.com/wiki/Rhaenyra_Targaryen",
    "https://gameofthrones.fandom.com/wiki/Viserys_I_Targaryen",
    "https://gameofthrones.fandom.com/wiki/Daenerys_Targaryen",
]
query = "who is the brother of DAEMON?"
answer, sources = rag_pipeline(urls, query)
print(f"Answer: {answer}")
print(f"Sources: {sources}")

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Answer: Lord Corlys
Sources: {'https://gameofthrones.fandom.com/wiki/Viserys_I_Targaryen'}


In [12]:
# Import the necessary libraries
import psutil
import time
import threading

def log_resource_usage(interval=10):
    """Log the resource usage at the specified interval (in seconds)."""
    while True:
        # Get the current CPU and memory usage
        cpu_usage = psutil.cpu_percent(interval=1)
        memory_info = psutil.virtual_memory()

        # Log the usage
        print(f"CPU Usage: {cpu_usage}%")
        print(f"Memory Usage: {memory_info.percent}%")

        # Wait for the next interval
        time.sleep(interval)

# Start logging resource usage in a separate thread
threading.Thread(target=log_resource_usage, daemon=True).start()
